# Brendon McNulty — ML Final
**Project 7** · Chart a Straight Line (10.16) + Predict Avg High Temp in NYC in January (15.4)

Repo: (https://github.com/brendonmcnulty/datafun-07-ml)

This notebook follows the Module 7 Outline:
- **Part 1**: Chart a Straight Line
- **Part 2**: Data Acquisition · Inspection · Cleaning · Descriptives
- **Part 3**: Build · Test · Predict (2024) · Visualize


In [ ]:
import warnings; warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# plotting defaults
plt.rcParams["figure.figsize"] = (7,5)
plt.rcParams["axes.grid"] = True

# data path from your repo
DATA_PATH = "examples/ch15/ave_hi_nyc_jan_1895-2018.csv"
RANDOM_STATE = 11


## Part 1 — Chart a Straight Line (10.16)

Refresh the linear equation **y = m x + b** with a simple Celsius→Fahrenheit line.


In [ ]:
# Celsius to Fahrenheit: F = 1.8*C + 32  (m=1.8, b=32)
c = np.arange(-20, 41, 5)
f = 1.8 * c + 32
cf_df = pd.DataFrame({"Celsius": c, "Fahrenheit": f})
display(cf_df.head())

ax = sns.lineplot(data=cf_df, x="Celsius", y="Fahrenheit", marker="o")
ax.set_title("Straight Line Demo: F = 1.8*C + 32")
plt.show()
